# Machine Learning - Training the models using a LSTM based network

First we are importing the necessary libraries and the datasets

In [1]:
import tensorflow as tf

from numpy import genfromtxt
import pandas as pd
from sklearn import preprocessing, model_selection
from tensorflow import keras
from tensorflow.metrics import precision
import matplotlib.pyplot as plt 
import os
from sklearn.metrics import precision_recall_fscore_support

features = genfromtxt('../datasets/final-data/features.csv', delimiter=',')
labels = genfromtxt('../datasets/final-data/labels.csv', delimiter=',')
#embedding_matrix = genfromtxt('../datasets/final-data/embedding_matrix.csv', delimiter=',')

train_x, test_x, train_y, test_y = model_selection.train_test_split(features,labels,test_size = 0.3, random_state = 0)

In [2]:
train_x.shape

(2982, 900)

# Creating the models to be tested

Now we are combining all the variable parameters and creating multiple models to be tested

In [3]:
# input_dim = len(embedding_matrix)
# input_length = len(train_x[0])

# Variable parameters to be tested
units_options = [150, 75, 35]
activation_functions = ["relu", "sigmoid", "tanh"]
learning_rates = [0.1, 0.01, 0.001, 0.0001]
momentum_options = [0.1, 0.01, 0.001, 0.0001]
decay_options = [0.1, 0.01, 0.001, 0.0001]

#Create multiple models combining all the parameters
models = []
for units in units_options:
    for learning_rate in learning_rates:
        for momentum in momentum_options:
            for activation_function in activation_functions:
                for decay in decay_options:

                    #Creating the network structure
                    model = keras.Sequential()

                    # Input, 
                    # Dense(linear(units = 150)), 
                    # Dense(relu), 
                    # batch_normalization(opcional, BatchNormalization1D), 
                    # linear(2)

                    model.add(
                        keras.layers.Input(
                            shape=900,
                            sparse=False
                        )
                    )


                    model.add(keras.layers.Dense(450))
                    model.add(keras.layers.Dense(units, activation = activation_function))
                    model.add(keras.layers.BatchNormalization())
                    model.add(keras.layers.Dense(2, activation='softmax'))

                    # Setting the optimizers parameters
                    optimizer = tf.keras.optimizers.SGD(
                        learning_rate=learning_rate,
                        decay=decay,
                        momentum=momentum,
                        nesterov=True
                    )

                    # Compiling the model
                    model.compile(
                        optimizer = optimizer,
                        loss='sparse_categorical_crossentropy',
                        metrics=['acc', 'mae', 'mse'])

                    # Including the new model in the models array
                    models.append(
                        {
                            "model_name": 'model_' + str(len(models) + 1),
                            "units": units, 
                            "learning_rate": learning_rate, 
                            "momentum": momentum,
                            "decay": decay,
                            "activation_function": 'relu',
                            "model": model
                        }
                    )


pd.DataFrame(models) 

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


,activation_function,decay,learning_rate,model,model_name,momentum,units
0,relu,0.1000,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_1,0.1000,150
1,relu,0.0100,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_2,0.1000,150
2,relu,0.0010,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_3,0.1000,150
3,relu,0.0001,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_4,0.1000,150
4,relu,0.1000,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_5,0.1000,150
5,relu,0.0100,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_6,0.1000,150
6,relu,0.0010,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_7,0.1000,150
7,relu,0.0001,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_8,0.1000,150
8,relu,0.1000,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_9,0.1000,150
9,relu,0.0100,0.1000,<tensorflow.python.keras.engine.sequential.Seq...,model_10,0.1000,150


# Training the models

Training all the created models and storing their performances

In [4]:
import numpy as np
training_results = []

trained_models = 1

for model_data in models:
    model = model_data["model"]

    early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

    history = model.fit(
        train_x, 
        train_y, 
        epochs = 50, 
        validation_split=0.3, 
        batch_size = 16,  
        verbose=1, 
        shuffle=True,
        callbacks=[early_stop]
    )

    loss, acc, mae, mse = model.evaluate(test_x, test_y, verbose=1)
    
    test_output_probabilities = model.predict(
        test_x,
        batch_size=16,
        verbose=1,
        steps=None,
        callbacks=None,
        max_queue_size=10,
        workers=1,
        use_multiprocessing=True
    )
    
    predicted_y = np.argmax(test_output_probabilities, axis=1)
    
    precision, recall, fs_score, support = precision_recall_fscore_support(
        y_true = test_y, 
        y_pred = predicted_y, 
        average = 'binary'
    )

    training_results.append(
        {
            "model_name": model_data["model_name"],
            "units": model_data["units"], 
            "learning_rate": model_data["learning_rate"], 
            "momentum": model_data["momentum"],
            "decay": model_data["decay"],
            "activation_function": model_data["activation_function"],
            "model": model,
            "history": history,
            "acc": acc,
            "loss": loss,
            "mae": mae,
            "mse": mse,
            "precision": precision,
            "recall": recall,
            "fs_score": fs_score,
            "test_output_probabilities": test_output_probabilities,
            "test_y": test_y,
            "predicted_y": predicted_y
        }
    )
    print("Trained models: {}".format(trained_models))
    trained_models = trained_models + 1

Train on 2087 samples, validate on 895 samples
Epoch 1/50
2087/2087 [==============================] - 11s 5ms/sample - loss: 0.7320 - acc: 0.5031 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2640 - val_loss: 0.7307 - val_acc: 0.4994 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2707acc: 0.4941 - mean_absolute_error: 0
Epoch 2/50
2087/2087 [==============================] - 2s 1ms/sample - loss: 0.6829 - acc: 0.5520 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2561 - val_loss: 0.6956 - val_acc: 0.5307 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2537
Epoch 3/50
2087/2087 [==============================] - 2s 1ms/sample - loss: 0.6773 - acc: 0.5644 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2560 - val_loss: 0.7070 - val_acc: 0.5017 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2598
Epoch 4/50
2087/2087 [==============================] - 2s 1ms/sample - loss: 0.6735 - acc: 0.5740 - mean_absolute_error: 0.5000 -

Epoch 11/50
2087/2087 [==============================] - 2s 1ms/sample - loss: 0.5929 - acc: 0.6814 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3046 - val_loss: 0.8495 - val_acc: 0.5151 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.3116
Epoch 12/50
1279/1279 [==============================] - 3s 2ms/sample
Trained models: 3
Train on 2087 samples, validate on 895 samples
Epoch 1/50
2087/2087 [==============================] - 10s 5ms/sample - loss: 0.8219 - acc: 0.5170 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2914 - val_loss: 3.5487 - val_acc: 0.5017 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.4970
Epoch 2/50
2087/2087 [==============================] - 2s 1ms/sample - loss: 0.7296 - acc: 0.5472 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2792 - val_loss: 0.7215 - val_acc: 0.5028 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2667
Epoch 3/50
2087/2087 [==============================] - 2s 1ms/sample - loss

2087/2087 [==============================] - 11s 5ms/sample - loss: 0.7561 - acc: 0.5194 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2707 - val_loss: 0.8648 - val_acc: 0.4972 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.3222
Epoch 2/50
2087/2087 [==============================] - 2s 1ms/sample - loss: 0.6819 - acc: 0.5764 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2669 - val_loss: 0.7574 - val_acc: 0.5028 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2817
Epoch 3/50
2087/2087 [==============================] - 2s 1ms/sample - loss: 0.6605 - acc: 0.6047 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2732 - val_loss: 0.7291 - val_acc: 0.5006 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2684
Epoch 4/50
2087/2087 [==============================] - 2s 1ms/sample - loss: 0.6348 - acc: 0.6339 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2817 - val_loss: 3.4673 - val_acc: 0.4972 - val_mean_absolute_error: 0.

C:\Users\gusta\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 1/50
2087/2087 [==============================] - 11s 5ms/sample - loss: 0.7282 - acc: 0.4983 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2620 - val_loss: 1.1272 - val_acc: 0.4972 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.3941
Epoch 2/50
2087/2087 [==============================] - 2s 1ms/sample - loss: 0.6761 - acc: 0.5798 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2554 - val_loss: 0.9068 - val_acc: 0.4972 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.3367
Epoch 3/50
2087/2087 [==============================] - 2s 1ms/sample - loss: 0.6681 - acc: 0.5951 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2566 - val_loss: 0.7223 - val_acc: 0.4972 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2658
Epoch 4/50
2087/2087 [==============================] - 2s 1ms/sample - loss: 0.6615 - acc: 0.6033 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2581 - val_loss: 0.7195 - val_acc: 0.5151 - val_mean_absolut

...

2087/2087 [==============================] - 6s 3ms/sample - loss: 0.8330 - acc: 0.5189 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3077 - val_loss: 0.8916 - val_acc: 0.4782 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.3113
Epoch 6/50
1279/1279 [==============================] - 11s 8ms/sample
Trained models: 556
Train on 2087 samples, validate on 895 samples
Epoch 1/50
2087/2087 [==============================] - 30s 14ms/sample - loss: 0.8789 - acc: 0.4935 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3088 - val_loss: 0.7093 - val_acc: 0.5028 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2573
Epoch 2/50
2087/2087 [==============================] - 6s 3ms/sample - loss: 0.8887 - acc: 0.4878 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3090 - val_loss: 0.7053 - val_acc: 0.4782 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2542
Epoch 3/50
2087/2087 [==============================] - 6s 3ms/sample - loss: 0.8831 

2087/2087 [==============================] - 6s 3ms/sample - loss: 0.7941 - acc: 0.5084 - mean_absolute_error: 0.5000 - mean_squared_error: 0.2921 - val_loss: 0.7898 - val_acc: 0.5073 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2900
Epoch 6/50
1279/1279 [==============================] - 11s 9ms/sample
Trained models: 560
Train on 2087 samples, validate on 895 samples
Epoch 1/50
2087/2087 [==============================] - 30s 14ms/sample - loss: 1.0900 - acc: 0.5026 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3550 - val_loss: 0.7724 - val_acc: 0.4961 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2839
Epoch 2/50
2087/2087 [==============================] - 6s 3ms/sample - loss: 1.0779 - acc: 0.5065 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3532 - val_loss: 0.7704 - val_acc: 0.5106 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2796
Epoch 3/50
2087/2087 [==============================] - 6s 3ms/sample - loss: 1.0716 

1279/1279 [==============================] - 11s 8ms/sample
Trained models: 564
Train on 2087 samples, validate on 895 samples
Epoch 1/50
2087/2087 [==============================] - 30s 14ms/sample - loss: 0.8639 - acc: 0.4907 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3090 - val_loss: 0.7599 - val_acc: 0.5073 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2837
Epoch 2/50
2087/2087 [==============================] - 6s 3ms/sample - loss: 0.8414 - acc: 0.5084 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3051 - val_loss: 0.7327 - val_acc: 0.5073 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2729
Epoch 3/50
2087/2087 [==============================] - 6s 3ms/sample - loss: 0.8369 - acc: 0.4959 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3037 - val_loss: 0.7673 - val_acc: 0.5218 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2868
Epoch 4/50
2087/2087 [==============================] - 6s 3ms/sample - loss: 0.8397 

2087/2087 [==============================] - 6s 3ms/sample - loss: 0.9523 - acc: 0.4696 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3222 - val_loss: 0.7161 - val_acc: 0.5017 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2591
Epoch 3/50
2087/2087 [==============================] - 6s 3ms/sample - loss: 0.9563 - acc: 0.4748 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3226 - val_loss: 0.7308 - val_acc: 0.4760 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2635
Epoch 4/50
2087/2087 [==============================] - 7s 3ms/sample - loss: 0.9659 - acc: 0.4605 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3227 - val_loss: 0.7861 - val_acc: 0.4771 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2820
Epoch 5/50
2087/2087 [==============================] - 6s 3ms/sample - loss: 0.9527 - acc: 0.4705 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3219 - val_loss: 0.8724 - val_acc: 0.4771 - val_mean_absolute_error: 0.5

2087/2087 [==============================] - 30s 14ms/sample - loss: 1.0330 - acc: 0.4887 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3396 - val_loss: 0.7400 - val_acc: 0.4983 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2707
Epoch 2/50
2087/2087 [==============================] - 6s 3ms/sample - loss: 1.0165 - acc: 0.4978 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3391 - val_loss: 0.7491 - val_acc: 0.5061 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2740
Epoch 3/50
2087/2087 [==============================] - 6s 3ms/sample - loss: 1.0242 - acc: 0.4931 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3376 - val_loss: 0.8171 - val_acc: 0.5050 - val_mean_absolute_error: 0.5000 - val_mean_squared_error: 0.2967
Epoch 4/50
2087/2087 [==============================] - 6s 3ms/sample - loss: 1.0325 - acc: 0.4964 - mean_absolute_error: 0.5000 - mean_squared_error: 0.3379 - val_loss: 0.9056 - val_acc: 0.5073 - val_mean_absolute_error: 0

# Creating a dataframe with the training metrics

We are going to store those metrics in a separate file

In [5]:
training_metrics_df = pd.DataFrame(training_results)

training_metrics_df = training_metrics_df[['model_name', 'units', 'learning_rate', 'momentum', 'decay', 'activation_function', 'acc', 'loss', 'mae', 'mse', 'precision', 'recall', 'fs_score']]

training_metrics_df.head(10)

,model_name,units,learning_rate,momentum,decay,activation_function,acc,loss,mae,mse,precision,recall,fs_score
0,model_1,150,0.1,0.1,0.1000,relu,0.506646,0.708369,0.5,0.258645,0.506073,0.392465,0.442087
1,model_2,150,0.1,0.1,0.0100,relu,0.533229,0.877448,0.5,0.314956,0.521978,0.745683,0.614092
2,model_3,150,0.1,0.1,0.0010,relu,0.499609,1.044821,0.5,0.355669,0.498706,0.907378,0.643653
3,model_4,150,0.1,0.1,0.0001,relu,0.527756,0.827854,0.5,0.309151,0.537585,0.370487,0.438662
4,model_5,150,0.1,0.1,0.1000,relu,0.507428,0.722504,0.5,0.267642,0.504993,0.555730,0.529148
5,model_6,150,0.1,0.1,0.0100,relu,0.499609,1.216112,0.5,0.391058,0.466667,0.032967,0.061584
6,model_7,150,0.1,0.1,0.0010,relu,0.512119,0.892059,0.5,0.320269,0.519174,0.276295,0.360656
7,model_8,150,0.1,0.1,0.0001,relu,0.501955,3.543080,0.5,0.499096,0.000000,0.000000,0.000000
8,model_9,150,0.1,0.1,0.1000,relu,0.512901,0.719480,0.5,0.267181,0.508065,0.692308,0.586047
9,model_10,150,0.1,0.1,0.0100,relu,0.497263,2.289862,0.5,0.479362,0.497653,0.998430,0.664230


# Saving the general experiments files

Saving the files with general data about the experiment:

- All datasets used(train_x, train_y, test_x, test_y)
- The training metrics of the models

In [6]:
np.savetxt("../experiments/pmean_body/train_x.csv", train_x, delimiter=",")
np.savetxt("../experiments/pmean_body/train_y.csv", train_y, delimiter=",")
np.savetxt("../experiments/pmean_body/test_x.csv", test_x, delimiter=",")
np.savetxt("../experiments/pmean_body/test_y.csv", test_y, delimiter=",")
training_metrics_df.to_csv('../experiments/pmean_body/training_metrics.csv', index= False, encoding='utf-8')

# Saving specific metrics of each model

For each trained model we are going to save:

- Predicted labels over the test_x
- The output probability on the test_x prediction
- The history of the training
- The trained model

In [7]:
for training_result in training_results:
    # Created the model folder
    model_folder = "../experiments/pmean_body/trained_models/" + training_result["model_name"]
    if not os.path.exists(model_folder):
        os.mkdir(model_folder)
    
    # Saves the predicted labels and the predicted probabilities on the test_x predictions
    np.savetxt(model_folder + "/predicted_y.csv", training_result["predicted_y"], delimiter=",")
    np.savetxt(model_folder + "/test_output_probabilities.csv", training_result["test_output_probabilities"], delimiter=",")
    
    # Creates a history dataframe and saves it in a file
    history_df = pd.DataFrame(training_result["history"].history)
    history_df['epoch'] = training_result["history"].epoch

    history_df.to_csv(model_folder + "/history.csv", index= False, encoding='utf-8')
    
    # Saves the trained model in a file
    training_result["model"].save(filepath=model_folder + "/model.hdf5", overwrite=True, include_optimizer=True)